In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import sys
import os
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *
from data_manager import *
from metadata import *

from vis3d_utilities import *

Setting environment for Local Macbook Pro


In [2]:
stack = 'MD589'
atlas_name = 'atlasV2'
classifier_setting = 2

In [3]:
# Load global transformed atlas structure meshes

global_transformed_atlas_polydata_list = \
DataManager.load_transformed_volume_meshes(stack_m=atlas_name, type_m='score', 
                                             stack_f=stack, type_f='score', 
                                           classifier_setting_m=classifier_setting,
                                           classifier_setting_f=classifier_setting,
                                           warp_setting=1, sided=True)

load_mesh_stl: File does not exist
Mesh is empty.
Error loading mesh for sp5.
load_mesh_stl: File does not exist
Mesh is empty.
Error loading mesh for outerContour.


In [4]:
# Load locally transformed atlas structure meshes

local_transformed_atlas_polydata_list = \
DataManager.load_transformed_volume_meshes(stack_m=atlas_name, type_m='score', 
                                           stack_f=stack, type_f='score', 
                                           classifier_setting_m=classifier_setting,
                                           classifier_setting_f=classifier_setting,
                                           warp_setting=2, sided=True)

load_mesh_stl: File does not exist
Mesh is empty.
Error loading mesh for sp5.
load_mesh_stl: File does not exist
Mesh is empty.
Error loading mesh for outerContour.


In [5]:
centroid_localAdjusted_polydata_list = {}
for name_s, polydata in local_transformed_atlas_polydata_list.iteritems():
    vertices, faces = polydata_to_mesh(polydata)
    centroid_localAdjusted_polydata_list[name_s] = vertices.mean(axis=0)

In [6]:
hessians_all_stacks_all_structures = defaultdict(dict)
for structure in all_known_structures_sided:
    try:
        hessians = DataManager.load_confidence(stack_m=atlas_name, stack_f=stack, 
                                classifier_setting_m=classifier_setting,
                               classifier_setting_f=classifier_setting,
                               warp_setting=2, what='hessians', param_suffix=structure)
        hessians_all_stacks_all_structures[stack][structure] = hessians
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error loading confidence: %s.\n' % structure)
        
hessians_all_stacks_all_structures.default_factory = None

[Errno 2] No such file or directory: '/home/yuncong/CSHL_registration_parameters/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_2_MD589_down32_scoreVolume_clf_2_trial_0/atlasV2_down32_scoreVolume_clf_2_warp_2_MD589_down32_scoreVolume_clf_2_trial_0_parameters_sp5_hessians.pkl'
Error loading confidence: sp5.
[Errno 2] No such file or directory: '/home/yuncong/CSHL_registration_parameters/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_2_MD589_down32_scoreVolume_clf_2_trial_0/atlasV2_down32_scoreVolume_clf_2_warp_2_MD589_down32_scoreVolume_clf_2_trial_0_parameters_outerContour_hessians.pkl'
Error loading confidence: outerContour.


In [7]:
hessians_all_structures_all_stacks = defaultdict(dict)
for st, x_per_stack in hessians_all_stacks_all_structures.iteritems():
    for struct, hessians in x_per_stack.iteritems():
        hessians_all_structures_all_stacks[struct][st] = hessians

In [8]:
zscores_all_stacks_all_structures = defaultdict(dict)
for structure in all_known_structures_sided:
    try:
        zscores = DataManager.load_confidence(stack_m=atlas_name, stack_f=stack, 
                                classifier_setting_m=classifier_setting,
                               classifier_setting_f=classifier_setting,
                               warp_setting=2, what='zscores', param_suffix=structure)
        zscores_all_stacks_all_structures[stack][structure] = zscores
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error loading confidence: %s.\n' % structure)    
        
zscores_all_stacks_all_structures.default_factory = None

[Errno 2] No such file or directory: '/home/yuncong/CSHL_registration_parameters/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_2_MD589_down32_scoreVolume_clf_2_trial_0/atlasV2_down32_scoreVolume_clf_2_warp_2_MD589_down32_scoreVolume_clf_2_trial_0_parameters_sp5_zscores.pkl'
Error loading confidence: sp5.
[Errno 2] No such file or directory: '/home/yuncong/CSHL_registration_parameters/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_2_MD589_down32_scoreVolume_clf_2_trial_0/atlasV2_down32_scoreVolume_clf_2_warp_2_MD589_down32_scoreVolume_clf_2_trial_0_parameters_outerContour_zscores.pkl'
Error loading confidence: outerContour.


In [9]:
zscores_all_structures_all_stacks = defaultdict(dict)
for st, x_per_stack in zscores_all_stacks_all_structures.iteritems():
    for struct, zscores in x_per_stack.iteritems():
        zscores_all_structures_all_stacks[struct][st] = zscores

# VTK sphere plot

In [10]:
pooling_radius_list = sorted(zscores_all_structures_all_stacks.values()[0].values()[0].keys())
print 'pooling radius:', pooling_radius_list
hessian_stepsize_list = sorted(hessians_all_structures_all_stacks.values()[0].values()[0].keys())
print 'hessian stepsize:', hessian_stepsize_list

pooling radius: [25.0, 118.75, 212.5, 306.25, 400.0]
hessian stepsize: [14.720000000000001, 84.640000000000001, 154.56, 224.48000000000002, 294.40000000000003]


In [11]:
pool_radius = pooling_radius_list[1]
hess_stepsize = hessian_stepsize_list[1]

In [12]:
stds_all_structures_all_stacks = {struct: {st: zscores[pool_radius][3] for st, zscores in x_per_struct.iteritems()} 
                                for struct, x_per_struct in zscores_all_structures_all_stacks.iteritems()}

In [13]:
length_factor = 1e6

R = defaultdict(dict)
V = defaultdict(dict)
S = defaultdict(dict)
Htr = defaultdict(dict)

for li, (name_s, hs) in enumerate(hessians_all_structures_all_stacks.iteritems()):

    for st, hessians in hs.iteritems():
        
        h_tr, _ = hessians[hess_stepsize]
        
        _, _, _, std = zscores_all_structures_all_stacks[name_s][st][pool_radius]
                
        Htr[name_s][st] = h_tr
        
        s, v = np.linalg.eigh(h_tr)
                
        S[name_s][st] = 1./2. * 10**2 * s[2] / std # eigenvalue of hessian
        V[name_s][st] = v # eigenvector of hessian

        R[name_s][st] = v*2e-9/s[None, :] * length_factor # each column is an axis of the ellipsoid
        R[name_s][st] = R[name_s][st] * np.sign(R[name_s][st][0])
        
R.default_factory = None
V.default_factory = None
S.default_factory = None
Htr.default_factory = None

# For one structure

In [94]:
# Show heat sphere for one structure.

mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in local_transformed_atlas_polydata_list.iteritems() 
               if name_s == '7N_L']

sphere_actors = [actor_mesh(polydata_heat_sphere(func=lambda v:\
                -1./2. * 10**2 * np.dot(np.dot(v, Htr[name_s][stack]), v) / stds_all_structures_all_stacks[name_s][stack],
                                    loc=centroid_localAdjusted_polydata_list[name_s], radius=20))
           for name_s in all_known_structures_sided if name_s == '7N_L']

launch_vtk(mesh_actors + sphere_actors)

In [55]:
# Compute the anchor point and three axis vectors for one structure.

anchor_points_and_axis_vectors = {name_s: (centroid_localAdjusted_polydata_list[name_s], \
    R[name_s][stack][:,2],
    R[name_s][stack][:,1],
    R[name_s][stack][:,0])
for name_s in all_known_structures_sided if name_s in ['VLL_R']}

In [56]:
# Show uncertainty ellipse for one structure.

mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in local_transformed_atlas_polydata_list.iteritems()
              if name_s in ['VLL_R']]

ellipse_actors = [actor_ellipse(anchor_point, anchor_vector0, anchor_vector1, anchor_vector2, wireframe=True)
                  for anchor_point, anchor_vector0, anchor_vector1, anchor_vector2 \
                  in anchor_points_and_axis_vectors.itervalues()]

launch_vtk(mesh_actors + ellipse_actors, init_angle='30')

In [36]:
# Show uncertainty arrows for one structure.

mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in local_transformed_atlas_polydata_list.iteritems() 
               if name_s in ['VLL_R']]

arrow_actors = list(chain(*[actor_arrows(anchor_point, anchor_vector0, anchor_vector1, anchor_vector2)
                  for anchor_point, anchor_vector0, anchor_vector1, anchor_vector2 \
                   in anchor_points_and_axis_vectors.itervalues()]))

launch_vtk(mesh_actors + arrow_actors, init_angle='30')

# For all structures

In [211]:
# Show heat sphere for all structures.

mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in local_transformed_atlas_polydata_list.iteritems()]

sphere_actors = [actor_mesh(polydata_heat_sphere(lambda v: -1./2. * 10**2 * np.dot(np.dot(v, Htr[name_s][stack]), v) / stds_all_structures_all_stacks[name_s][stack], 
                                            loc=centroid_localAdjusted_polydata_list[name_s], radius=20))
           for name_s in all_known_structures_sided if name_s in centroid_localAdjusted_polydata_list]

launch_vtk(mesh_actors + sphere_actors)

In [38]:
# Compute the anchor point and three axis vectors for all structures.

anchor_points_and_axis_vectors = {name_s: (centroid_localAdjusted_polydata_list[name_s], \
    R[name_s][stack][:,2],
    R[name_s][stack][:,1],
    R[name_s][stack][:,0])
for name_s in all_known_structures_sided if name_s in R}

In [39]:
# Show uncertainty ellipse for all structures.

mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in local_transformed_atlas_polydata_list.iteritems()]

ellipse_actors = [actor_ellipse(anchor_point, anchor_vector0, anchor_vector1, anchor_vector2, wireframe=True)
                  for anchor_point, anchor_vector0, anchor_vector1, anchor_vector2 \
                  in anchor_points_and_axis_vectors.itervalues()]

launch_vtk(mesh_actors + ellipse_actors, init_angle='30')

In [50]:
# Show uncertainty arrows for all structures.

mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in local_transformed_atlas_polydata_list.iteritems()]

arrow_actors = list(chain(*[actor_arrows(anchor_point, anchor_vector0, anchor_vector1, anchor_vector2)
                  for anchor_point, anchor_vector0, anchor_vector1, anchor_vector2 \
                   in anchor_points_and_axis_vectors.itervalues()]))

launch_vtk(mesh_actors + arrow_actors, init_angle='30')

# For Yoav

In [14]:
high_contrast_colors = np.random.permutation(high_contrast_colors)
map_name_s_to_color = {s: np.array(high_contrast_colors[i%len(high_contrast_colors)]) 
                     for i, s in enumerate(all_known_structures_sided) }

In [15]:
map_name_u_to_color = {s: np.array(high_contrast_colors[i%len(high_contrast_colors)]) 
                     for i, s in enumerate(all_known_structures) }

In [16]:
tb_vol_xmin, tb_vol_xmax, tb_vol_ymin, tb_vol_ymax, tb_vol_zmin, tb_vol_zmax = (0, 484, 0, 372, 123, 503)

In [17]:
crop_xmin, crop_xmax, crop_ymin, crop_ymax = metadata_cache['cropbox'][stack]
print 'crop:', crop_xmin, crop_xmax, crop_ymin, crop_ymax

crop: 569 1053 140 512


In [18]:
shell_polydata = DataManager.load_shell_mesh(stack='MD589', downscale=32)
shell_actor = actor_mesh(shell_polydata, (1,1,1), opacity=.15, wireframe=False)

In [19]:
certain_regions = ['7N_L', '7N_R', '5N_L', '5N_R', '12N', 'Pn_L', 'Pn_R', 'LC_L', 'LC_R', 'Amb_L', 'Amb_R',
               'Tz_L', 'Tz_R', '3N_L', '3N_R', '4N_L', '4N_R', 'RMC_L', 'RMC_R', '10N_L', '10N_R']

In [217]:
# Panel 1

map_name_to_opacity = {name: .5 if name in certain_regions else 0.05
                      for name in all_known_structures_sided}

anchor_points_and_axis_vectors = {name_s: (centroid_localAdjusted_polydata_list[name_s], \
    R[name_s][stack][:,2],
    R[name_s][stack][:,1],
    R[name_s][stack][:,0])
for name_s in all_known_structures_sided if name_s in R}

mesh_actors = [actor_mesh(polydata, map_name_u_to_color[convert_to_original_name(name_s)]/255., 
                          opacity=map_name_to_opacity[name_s], 
                          wireframe=False,
                         origin=(crop_xmin + tb_vol_xmin, crop_ymin + tb_vol_ymin, tb_vol_zmin)) 
               for name_s, polydata in global_transformed_atlas_polydata_list.iteritems()]

launch_vtk([shell_actor] + mesh_actors, init_angle='30')

In [ ]:
# Panel 2

map_name_to_opacity = {name: .5 if name in certain_regions else 0.0
                      for name in all_known_structures_sided}

mesh_actors = [actor_mesh(polydata, map_name_u_to_color[convert_to_original_name(name_s)]/255., 
                          opacity=map_name_to_opacity[name_s], 
                          wireframe=False,
                         origin=(crop_xmin + tb_vol_xmin, crop_ymin + tb_vol_ymin, tb_vol_zmin)) 
               for name_s, polydata in global_transformed_atlas_polydata_list.iteritems()]

launch_vtk([shell_actor] + mesh_actors, init_angle='30')

In [20]:
# Panel 3

structures_to_show = ['VLL_L', 'Sp5O_L' ]

def f(name):
    if name in certain_regions:
        return .04
    elif name in structures_to_show:
        return 1.
    else:
        return 0

mesh_actors = [actor_mesh(polydata, map_name_s_to_color[name_s]/255., 
                          opacity=f(name_s),
                          wireframe=False,
                         origin=(crop_xmin + tb_vol_xmin, crop_ymin + tb_vol_ymin, tb_vol_zmin)) 
               for name_s, polydata in local_transformed_atlas_polydata_list.iteritems()]

launch_vtk([shell_actor] + mesh_actors, init_angle='30')

In [29]:
map_name_s_to_color = {'Sp5O_L': (1,0,0), 'VLL_L': (1,1,0)}

In [ ]:
# Panel 4

anchor_points_and_axis_vectors = {name_s: (centroid_localAdjusted_polydata_list[name_s], \
    R[name_s][stack][:,2],
    R[name_s][stack][:,1],
    R[name_s][stack][:,0])
for name_s in all_known_structures_sided if name_s in structures_to_show}


ellipse_actors = [actor_ellipse(anchor_point + (crop_xmin + tb_vol_xmin, crop_ymin + tb_vol_ymin, tb_vol_zmin), 
                                anchor_vector0, anchor_vector1, anchor_vector2, wireframe=True, opacity=1.0,
                               color=map_name_s_to_color[name_s])
                  for name_s, (anchor_point, anchor_vector0, anchor_vector1, anchor_vector2) \
                  in anchor_points_and_axis_vectors.iteritems()]

launch_vtk([shell_actor] + ellipse_actors, init_angle='30')